In [1]:
import numpy as np
import pandas as pd
import json
import re

In [2]:
resData = pd.read_csv('./Datasets/Cleaned Grab SG Restaurants.csv')
resData.head()


,name,address,cuisine,delivery_cost,lat,lon,opening_hours,radius,rating,reviews_nr,delivery_options,promo,loc_type,delivery_by,delivery_time,total opening time hours
0,Texas Chicken,Texas Chicken - VivoCity,"['""Western""', '""Fast Food""', '""Halal""', '""Chic...",570.0,1.264792,103.822103,"{""open"": true, ""displayedHours"": ""10:30-21:30""...",5000,3.8,1591.0,ONLY_DELIVERY,Use 'TEATIME' for FREE delivery between 3PM to...,FOOD,GRAB,43.0,75.75
1,Subway,Subway - Vivocity,"['""Healthy""', '""Sandwiches""', '""Fast Food""', '...",570.0,1.264741,103.822072,"{""open"": true, ""displayedHours"": ""12:00-19:00""...",3000,4.2,197.0,ONLY_DELIVERY,Use 'TEATIME' for FREE delivery between 3PM to...,FOOD,GRAB,42.0,49.00
2,LiHO Tea,LiHO Tea - Vivo City,"['""Same Prices In-Store""', '""Beverages""', '""Lo...",560.0,1.264309,103.821369,"{""open"": true, ""displayedHours"": ""11:30-20:45""...",5000,4.5,353.0,DELIVERY_TAKEAWAY,Use 'TEATIME' for FREE delivery between 3PM to...,FOOD,GRAB,41.0,64.75
3,Wingstop,Wingstop - VivoCity,"['""Halal""', '""Chicken""', '""Fast Food""', '""Burg...",560.0,1.263527,103.821712,"{""open"": true, ""displayedHours"": ""11:00-21:00""...",5000,4.4,1391.0,ONLY_DELIVERY,Use 'TEATIME' for FREE delivery between 3PM to...,FOOD,GRAB,45.0,70.00
4,Le Shrimp Ramen,Le Shrimp Ramen - Vivocity,"['""Japanese""', '""Ramen""', '""Noodles""', '""Asian""']",570.0,1.264810,103.822554,"{""open"": true, ""displayedHours"": ""10:30-21:00""...",5000,4.6,309.0,DELIVERY_TAKEAWAY,Use 'TEATIME' for FREE delivery between 3PM to...,FOOD,GRAB,44.0,73.50


# Calculate weighted score

In [3]:
def rank_cuisine_by_weighted_average(df):
    # Drop rows with missing values in the 'cuisine' or 'rating' columns
    df = df.dropna(subset=['cuisine', 'rating'])
    
    # Initialize dictionary to store cuisine ratings
    cuisine_ratings = {}
    
    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        # Split the cuisine tags string by comma and remove square brackets
        tags_list = [tag.strip().strip('"') for tag in row['cuisine'].strip('[]').split(',') if tag.strip()]
        
        # Iterate over each cuisine tag
        for tag in tags_list:
            # If the tag is not in the dictionary, add it with an empty list
            if tag not in cuisine_ratings:
                cuisine_ratings[tag] = {'ratings': [], 'count': 0}
            # Append the rating to the list of ratings for this cuisine tag
            cuisine_ratings[tag]['ratings'].append(row['rating'])
            # Increment the rating count for this cuisine tag
            cuisine_ratings[tag]['count'] += 1
    
    # Calculate the weighted average for each cuisine tag
    weighted_averages = []
    total_ratings = sum(sum(ratings_info['ratings']) for ratings_info in cuisine_ratings.values())
    for tag, ratings_info in cuisine_ratings.items():
        scores_sum = sum(ratings_info['ratings'])
        num_ratings = ratings_info['count']
        weighted_average = (scores_sum * num_ratings) / total_ratings
        weighted_averages.append({'Tag': tag, 'Weighted Average': weighted_average, 'Rating Count': num_ratings})
    
    # Create DataFrame from the list of weighted averages
    df_weighted_averages = pd.DataFrame(weighted_averages)
    
    # Sort DataFrame by weighted average in descending order
    df_weighted_averages = df_weighted_averages.sort_values(by='Weighted Average', ascending=False)
    
    return df_weighted_averages

In [6]:
# Get ranked cuisine tags by weighted average
ranked_cuisine_weighted_avg_df = rank_cuisine_by_weighted_average(resData)

# Display the DataFrame
ranked_cuisine_weighted_avg_df.head()

,Tag,Weighted Average,Rating Count
8,"'""Local""'",314.432006,3860
14,"'""Asian""'",309.845949,3839
22,"'""Chinese""'",253.096458,3483
0,"'""Western""'",104.014970,2220
13,"'""Noodles""'",79.774565,1952


<font size="3">From the data, we see that 'Local' is the most popular cuisine tag. We use <b>weighted scoring</b> to blend the average rating and the number of ratings. This helps us get a better picture of popularity by considering both the <b>quality</b> and <b>quantity</b> of reviews. It shows which cuisine tags get high ratings and lots of attention from customers.</font>


In [5]:
ranked_cuisine_weighted_avg_df.to_csv('./Datasets/ranked_cuisine_df.csv', index=False)